This notebook shows you how to load a multi-page PDF, tag each page with useful metadata like doc_type and page_number, and build a smart index that lets you retrieve only the content you need using filters.



In [1]:
#Download the necessary packages
!pip install llama-index
!pip install llama-index-readers-file
!pip install llama-index llama-index-embeddings-huggingface transformers sentence-transformers

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**1. Loading a multi-page PDF**

In [2]:
from llama_index.readers.file import PDFReader

loader = PDFReader()
pages = loader.load_data("/content/sample_data/Blob File Sample.pdf")  # Returns one Document per page

# Print preview
print(f"Loaded {len(pages)} pages")
print(pages[0].text[:300])

Loaded 4 pages
Functional Resume Sample 
 
John W. Smith  
2002 Front Range Way Fort Collins, CO 80525  
jwsmith@colostate.edu 
 
Career Summary 
 
Four years experience in early childhood development with a diverse background in the care of 
special needs children and adults.  
  
Adult Care Experience 
 
• Deter


**2. Adding metadata like doc_type and page_number**

In [3]:
documents = []
for i, doc in enumerate(pages):
    doc.metadata = {
        "page_number": i + 1,
        "source_file": "sample_blob.pdf"
    }
    documents.append(doc)


In [4]:
doc_type_array = ["Resume", "Resume", "Lender Fees", "Contract", "Contract"]

for doc, doc_type in zip(documents, doc_type_array):
    doc.metadata["doc_type"] = doc_type


In [5]:
for doc in documents[:2]:
    print(doc.metadata)
    print(doc.text[:150], "\n---\n")

{'page_number': 1, 'source_file': 'sample_blob.pdf', 'doc_type': 'Resume'}
Functional Resume Sample 
 
John W. Smith  
2002 Front Range Way Fort Collins, CO 80525  
jwsmith@colostate.edu 
 
Career Summary 
 
Four years experi 
---

{'page_number': 2, 'source_file': 'sample_blob.pdf', 'doc_type': 'Resume'}
Your actual rate, payment, and cost could be higher. Get an official Loan Estimate before choosing a loan.
Fee Details and Summary
Applicants: Applica 
---



**3. Indexing and retrieving with metadata filters**

In [6]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Use a local embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

# Build a vector index enriched with metadata (doc_type, page_number, source_file)
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

print("Metadata-enriched index created with", len(documents), "documents.")



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Metadata-enriched index created with 4 documents.


In [7]:
# Query the full index
retriever = index.as_retriever()
query = "What information is provided on the employee's payslip?"
all_results = retriever.retrieve(query)

# Filter results using metadata (Lambda-style filtering)
filtered_results = [
    r for r in all_results if r.metadata.get("doc_type") == "Contract"
]

# Display the filtered results
for i, r in enumerate(filtered_results):
    print(f"--- Result {i+1} ---")
    print(r.text[:500])
    print("Metadata:", r.metadata)
    print("\n")


--- Result 1 ---
Payslip
Unknown and Co.
Pay Date : 2012/09/10
Working Days : 21
Employee Name : Joe Boe
Employee ID : 0211
Earnings Amount Deductions Amount
Basic Pay 3400 Tax 730
Allowance 500
Overtime 210
    
Total Earnings 4110 Total Deductions 730
  Net Pay 3380
3380
Three Thousand Three Hundred And Eighty
Employer Signature
_________________________________
Employee Signature
_________________________________
This is system generated payslip
Metadata: {'page_number': 4, 'source_file': 'sample_blob.pdf', 'doc_type': 'Contract'}


